# TME 4 : Suffix Trees


## Recheche de pattern (motifs) en utilisant les suffix trees

Nous allons utiliser l'algorithme suffix-tree pour une recherche rapide et éfficace de motifs. Un suffix-tree est construit à partir d'un jeux de séquences, ensuite nous pouvons rechercher le motif en temps O(k) où k est la longueur du motif.

1\. Nous allons réutiliser les fonctions du TMEs precedents pour générer ``t`` séquences artificielles de taille ``n``, et implanter dans chaque séquence un motif de taille ``k`` à des positions aléatoires avec ``v`` substitutions choisies aléatoirement. Cependant, les ``t`` séquences artificielles initiales (sans implantation) ainsi que le motif initial (sans variation/mutation) doivent être générées une seule fois. Ensuite, selon chaque question, nous introduisons des différentes variation au motif initial et les implantons dans les séquences initiales afin de générer des nouveau jeux de données. 

In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=8 #taille de motif
v=0 #nb de positions variables dans le motif
t=5 #nb de sequences
n=100 #longuer des sequence
f=0.9


1.1\. Generer les séquences artificielles initiales et implanter un motif (sans variation, v=0)

In [54]:

nuc = ('A', 'C', 'G', 'T')

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Même chose hein
    """
    sequence = "".join([random.choice(nuc) for _ in range(n)])
    if upper:
        return sequence
    return sequence.lower()

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    nbPos_real = min(nbpos, len(motif))
    allPos = list(range(len(motif)))

    for _ in range(nbPos_real):
        index = random.choice(range(len(allPos)))
        changeIndex = allPos[index]
        del allPos[index]

        nv_L = generateRandomSequence(1, upper)
        motifM[changeIndex] = nv_L
    return "".join(motifM)        


def implantMotifVar(adn, k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    ##adn false or true, parce que c'est pas grave pour notre TME si on travaille avec des sequences pregenerés ou on les genere ici
    
    if (not adn):
        adn = []
        for i in range(t):
            adn.append(generateRandomSequence(n - k))
    
    sequences = adn

    motif = generateRandomSequence(k)
    print(motif)
    motif_new = motif
    for i in range(t):
        base_seq = adn[i]
        if random.random() < f:
            motif_new = modifierMotif(motif, v)
        sequences.append(insertMotif(base_seq, motif_new, random.choice(range(len(base_seq)))))
        motif_new = motif
    
    return adn, sequences, motif_new


adn = implantMotifVar(False, k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

gatacacg


TypeError: 'bool' object is not subscriptable

2\. Définissez une fonction ``construct_tree`` pour construire un suffix tree à partir des séquences artificielles (après implantation) en utilisant le python package suffix-trees trouvable ici: https://pypi.org/project/suffix-trees/. Tester si votre fonction est capable de trouver le motif sans variation implanté.

In [3]:
!pip install suffix-trees

  Obtaining dependency information for suffix-trees from https://files.pythonhosted.org/packages/14/18/f252d1af9ff13a5bc94a4691450794603d4b6dfe99b257e1fb672082afe7/suffix_trees-0.3.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
from suffix_trees import STree

st = STree.STree("abcdefghab")
print(st.find("abc")) # 0
print(st.find_all("ab")) # [0, 8]

def construct_tree(sequences):
    """
    construire un abre de suffixe
    entrée1 : matrice de dimension txn avec les sequences
    sortie1 : arbre de suffixe
    REMARK: Vous devez concatener toutes les sequences de la matrice avant d'appeller la fonction STree
    """
    ns = ''.join(sequences[0:len(sequences)-1])
    st = STree.STree(ns)
    return st

tree = construct_tree(adn)
fix_motif = 'ACA' #changer pour un teste rapide
tree.find_all(fix_motif)


0
{8, 0}


{16, 24, 33, 173, 190, 257, 278, 319, 370, 372}

3\. Avant de chercher les motifs, implémentez ou reutilisez les fonctions pour générer tous les motifs (k-mer) possibles de taille k, en éliminant les motifs peu complexe pour éviter les calculs inutiles.

In [46]:
from itertools import product

def kmerList(allkmers):
    validKmers = []
    for i in allkmers:
        kmer = "".join(i)
        validKmers.append(kmer)
    return validKmers


def removeLowComplexe(motifs, size):
    """
    Eleve les motifs peu complexe ayant 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie motifsClean: dictionnaire de motifs sans les motifs peu complexe.
    """
    motifsClean = []
    twoLetterCombs = ["".join(tup) for tup in list(product(["A", "T", "G", "C"], repeat=2))]
    for motif in [m.upper() for m in motifs]:
        if motif.count("A") > size or motif.count("T") > size or motif.count("G") > size or motif.count("C") > size:
            continue
        if True in [x*3 in motif for x in twoLetterCombs]:
            print(motif)
            continue
        motifsClean.append(motif)
        
    return motifsClean



allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)

print (len(kmers))
kmersValid = removeLowComplexe(kmers, 5)
print (len(kmersValid))
print (kmersValid[0])

kmersValid = removeLowComplexe(kmersValid, 5)
print (len(kmersValid))

65536
AAACACAC
AAAGAGAG
AAATATAT
AACACACA
AACACACC
AACACACG
AACACACT
AACGCGCG
AACTCTCT
AAGAGAGA
AAGAGAGC
AAGAGAGG
AAGAGAGT
AAGCGCGC
AAGTGTGT
AATATATA
AATATATC
AATATATG
AATATATT
AATCTCTC
AATGTGTG
ACACACAA
ACACACAC
ACACACAG
ACACACAT
ACACACCA
ACACACCC
ACACACCG
ACACACCT
ACACACGA
ACACACGC
ACACACGG
ACACACGT
ACACACTA
ACACACTC
ACACACTG
ACACACTT
ACAGAGAG
ACATATAT
ACCACACA
ACCGCGCG
ACCTCTCT
ACGAGAGA
ACGCGCGA
ACGCGCGC
ACGCGCGG
ACGCGCGT
ACGTGTGT
ACTATATA
ACTCTCTA
ACTCTCTC
ACTCTCTG
ACTCTCTT
ACTGTGTG
AGACACAC
AGAGAGAA
AGAGAGAC
AGAGAGAG
AGAGAGAT
AGAGAGCA
AGAGAGCC
AGAGAGCG
AGAGAGCT
AGAGAGGA
AGAGAGGC
AGAGAGGG
AGAGAGGT
AGAGAGTA
AGAGAGTC
AGAGAGTG
AGAGAGTT
AGATATAT
AGCACACA
AGCGCGCA
AGCGCGCC
AGCGCGCG
AGCGCGCT
AGCTCTCT
AGGAGAGA
AGGCGCGC
AGGTGTGT
AGTATATA
AGTCTCTC
AGTGTGTA
AGTGTGTC
AGTGTGTG
AGTGTGTT
ATACACAC
ATAGAGAG
ATATATAA
ATATATAC
ATATATAG
ATATATAT
ATATATCA
ATATATCC
ATATATCG
ATATATCT
ATATATGA
ATATATGC
ATATATGG
ATATATGT
ATATATTA
ATATATTC
ATATATTG
ATATATTT
ATCACACA
ATCGCGCG
ATCTCTCA
ATCTCTCC
ATCTCTCG
ATCT

4\. **Exact matching:** Définissez la fonction ``exact_match`` qui cherche dans le suffix tree tous les motifs possibles (k-mers), générés à la question precedent. La fonction renvoie un dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrence (valeurs). Ce dictionnaire doit être trié par nombre d'occurrences. 

Ensuite, trouvez et affichez les 10 motifs plus fréquents dans notre jeux de données artificiels.

In [49]:
def exact_match(kmersV, stree):
    """
    Cherche dans le suffix tree tous les motifs possibles
    entrée1 kmersV: Liste de Kmers à chercher
    entrée2 stree: suffix tree
    sortie1 motif_occur_sorted: dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrences (values).
    """
    motif_occur_sorted = {}

    for i in kmersV:
        motif_occur_sorted[i] = len(stree.find_all(i))

    return dict(sorted(motif_occur_sorted.items(), key=lambda x: x[1], reverse=True))

st = construct_tree(adn)
motif_occur_sorted = exact_match(kmersValid, st)
motif_occur_sorted



{'ATACCCAC': 4,
 'AAAACGTG': 1,
 'AAAATTCA': 1,
 'AAACGTGT': 1,
 'AAAGCCCC': 1,
 'AAAGGTTA': 1,
 'AAATACCC': 1,
 'AAATACTT': 1,
 'AAATTCAG': 1,
 'AACAGCCT': 1,
 'AACGGGAC': 1,
 'AACGTGTT': 1,
 'AAGCCCCC': 1,
 'AAGGTTAG': 1,
 'AAGTAGTC': 1,
 'AAGTGGCG': 1,
 'AATACCCA': 1,
 'AATACTTA': 1,
 'AATCCGTA': 1,
 'AATTCAGT': 1,
 'AATTCCAA': 1,
 'ACAAAGGT': 1,
 'ACACATGG': 1,
 'ACACCGGT': 1,
 'ACAGATAC': 1,
 'ACAGCCTC': 1,
 'ACAGCGCT': 1,
 'ACAGCTGG': 1,
 'ACAGGTTC': 1,
 'ACATGGCT': 1,
 'ACATGTGG': 1,
 'ACCAACAG': 1,
 'ACCACACC': 1,
 'ACCAGCGG': 1,
 'ACCCACAG': 1,
 'ACCCACCA': 1,
 'ACCCACGA': 1,
 'ACCCACTG': 1,
 'ACCGCTTT': 1,
 'ACCGGTGA': 1,
 'ACGACGCG': 1,
 'ACGAGAAA': 1,
 'ACGCGATT': 1,
 'ACGGGACA': 1,
 'ACGTGTTC': 1,
 'ACGTTGGT': 1,
 'ACTAAATA': 1,
 'ACTGACCC': 1,
 'ACTGTCAG': 1,
 'ACTTACAT': 1,
 'ACTTCCGG': 1,
 'ACTTTGTT': 1,
 'AGAAAATT': 1,
 'AGACAAAG': 1,
 'AGATACCC': 1,
 'AGCAGGGC': 1,
 'AGCCCCCG': 1,
 'AGCCTCCG': 1,
 'AGCGCTAC': 1,
 'AGCGGCAT': 1,
 'AGCTGGCA': 1,
 'AGGGCTTC': 1,
 'AGGTTA

5\. Introduisez deux variation (v=2) au motif initial. Pour cela avant de chaque implantation, créez d'abord un motif varié (avec v substitutions choisies aléatoirement) à partir du motif initial et puis implantez-le dans une séquence. Repetez pour chaque sequence dans le Jeux de donnée. Il suffit de mettre ``v`` égal ``2`` et réutiliser les fonctions définies à la question 1.

In [50]:
v=2
adnOri, adn,  fix_motif = implantMotifVar(adnOri = False, fix_motif, k, v, t, n)


NameError: name 'implantMotif' is not defined

6\. Construisez le suffix tree à nouveau à partir des nouvelles séquences en utilisant le python package suffix-trees.

In [ ]:
st = construct_tree(adn)

7\. **Inexact matching:** 

Définissez fonction ``inexact_match`` qui cherche tous les motifs possibles (k-mers) générés à la question 2 dans le nouveau suffix tree donné (construit à partir des nouvelle séquences qui incluent le motif varié), et renvoie un dictionnaire qui contient les motifs (keys) et les listes de toutes leurs variations (values) ainsi que le meilleur motif variable. Il faut que vous utilisiez la technique *seed* pour trouver le motif variable. 

Ensuite, affichez le meilleur motif variable avec toutes son variation dans notre nouveaux jeux de données artificiels.
***Transformer en deux fonction separer, une pour for motif et autre for seed***

In [ ]:
def inexact_match(kmersV, sequences, stree, v):
    """
    cherche de motif variables dans un suffix tree
    entrée1 kmersV: liste de motifs à chercher
    entrée2 sequences: matrice de dimension txn avec les sequences
    entrée3 stree: suffix Tree
    entrée4 v: nombre de variations dans le motifs
    sortie1 motif_occur_sorted: dictionnaire clés=sequence consensus des motifs; value= nombre d'occurrences.
    """
    motif_occur_sorted = {}
  

    return motif_occur_sorted

#Test
seqTest = "banananabanbnaabanbna"
st = construct_tree([seqTest])
k=6
motif_occur_sorted = inexact_match(['banbna'], [seqTest], st, 1)


8\. Créez le motif logo à partir des séquences du meilleur motif variable que vouz venez de trouver. Vous pouvez utilizer ce site: https://weblogo.berkeley.edu/logo.cgi. Affichez votre logo ci-dessous.

9\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Quelle est la complexité de chaque recherche de motif?

Votre reponse

10\. Tester l'algorithme  suffix tree sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [ ]:
def readFasta(genome):
    sequence = []
    file = open(genome, "r")
    sequences = []
    seq = ""
    for s in file:
        if s[0] != ">":
            seq += s.strip().upper()
        else:
            sequences.append(seq)
            seq = ""
    return sequences[1:]
genome = "Sequence_by_Peaks_0.fasta"
sequences = readFasta(genome)
#print(sequences)